# Step 2: ANALYZE

In [74]:
import pandas as pd
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
import re
import pandas as pd
import numpy as np

In [162]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as SW_en
from spacy.lang.es.stop_words import STOP_WORDS as SW_es

In [76]:
#import nltk 
#from nltk.corpus import stopwords 
#set(stopwords.words('english'))


In [77]:
#!python -m spacy download en_core_web_md
#!python -m spacy download en_core_web_sm
#!python -m spacy download es_core_news_md

In [79]:
dic_key_words = \
{   'data analyst'  : ['data analyst','analisis de datos', 
                       'data analytics', 'analitica de datos',
                       'business analyst', 'analisis de negocios',
                       'sql',
                       'python',
                       'business intelligence', 'inteligencia de negocios' , ' bi ' , 
                       'excel', 'microsoft office', 'ms office' ,
                       'dashboard','kpi', 
                       'reporting', 'informes',
                       'tableau', 'powerbi', 'power bi', 'microsoft bi', 'qlik', 'spotfire',
                      ],
 
    'data scientist': ['data scientist', 'cientifico de datos',
                       'data science', 'ciencia de datos',
                       'sql', 
                       'python', ' r ', 
                       ' ml ', 'machine learning','deep learning','neuronal networks', 
                       'pandas', 'numpy', 'scipy', 'web scrapping' , 
                       'data visualization', 'visualizacion de datos','matplotlib', 'seaborn', 
                       'spss',  'statistics',
                       'sklearn','sk learn', 'sk-learn','scikit learn', 'scikit-learn' ,'tensorflow', 'keras',
                       'data wrangling', 'limpieza de datos',
                       ' nlp ', 'a/b testing','a b testing'
                      ],
 
    'data engineer':  ['data engineer', 'ingeniero de datos',
                       'sql', 'nosql', 'no sql', 'bases de datos no relacionales',
                       'python', ' r ', 'scala', 'java ', 'spark', 'hadoop', 'haddop' 'hive', 'impala', 'kafka', 
                       'data wrangling', 'limpieza de datos',
                       'etl','extract, transform and load','map reduce','mapreduce', 
                       'data acquisition', 'adquisicion de datos',
                       'big data', 'bigdata', 'bigquery',
                       'data warehouse', 
                       ' aws ', 'azure',
                       'software engineering'
                      ]
}

## Getting csv

#### > PROFILE:

In [116]:
if platform.system() == 'Linux':
    df_profile = pd.read_csv(f'../data/raw/df_profile_cristopherrojaslepe.csv',
                             sep = ';',
                             encoding= 'utf8'
                  )
elif platform.system() == 'Windows':
    df_profile = pd.read_csv(f'..\\data\\raw\\df_profile.csv',
                             sep = ';',
                             encoding= 'utf8'
    
df_profile

,Unnamed: 0,info
0,profile,cristopherrojaslepe
1,headline,data scientist & analyst | python | pandas | n...
2,open new jobs,True
3,about,data scientist & analyst and industrial engine...
4,experiences,"['collection coordination analyst', 'real esta..."
5,years of experiences,"['1 yr 2 mos', '1 yr', '2 yrs 1 mo', '8 mos', ..."
6,total years,7
7,degrees,"['master', 'engineering - be']"
8,fields,"['data analytics', 'industrial engineering']"
9,skills,"['numpy', 'mysql', 'business analysis', 'micro..."


In [117]:
## ONLY FOR JUPYTER !!!!!!!!!!
df_profile.rename(columns={'Unnamed: 0':'type'},inplace=True)
df_profile.set_index('type', inplace=True)

In [118]:
df_profile

,info
type,
profile,cristopherrojaslepe
headline,data scientist & analyst | python | pandas | n...
open new jobs,True
about,data scientist & analyst and industrial engine...
experiences,"['collection coordination analyst', 'real esta..."
years of experiences,"['1 yr 2 mos', '1 yr', '2 yrs 1 mo', '8 mos', ..."
total years,7
degrees,"['master', 'engineering - be']"
fields,"['data analytics', 'industrial engineering']"


#### > JOBS:

In [83]:
#f'..\\data\\raw\\df_jobs_Madrid_2020-04-18 23.50.csv'
if platform.system() == 'Linux':
    df_jobs_raw = pd.read_csv(f'../data/raw/df_jobs_Madrid_2020-04-18 23.50.csv',
                              sep = ';',
                              encoding= 'utf8',
                  )
elif platform.system() == 'Windows':
    df_jobs_raw = pd.read_csv(f'..\\data\\raw\\df_jobs_Madrid_2020-04-18 23.50.csv',
                              sep = ';',
                              encoding= 'utf8',
                  )

In [84]:
df_jobs = df_jobs_raw

In [85]:
df_jobs.shape

(859, 17)

In [86]:
df_jobs.head(3)

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,Easy apply,Skills match,Seniority Level,Industry,Employment Type,Job Functions,Job info
0,data analyst,Madrid,2020-04-18 23:50,1804937837,https://www.linkedin.com/jobs/view/1804937837/,Business Process Analyst,SWIFT,"Madrid, ES",1 day,2020-04-17 23:50,False,"{'yes': ['Portfolio Management', 'Business Int...",Associate,Information Technology & Services Computer Sof...,Full-time,Management Manufacturing,Posted Date\nPosted 1 day ago\nNumber of views...
1,data analyst,Madrid,2020-04-18 23:50,1828401888,https://www.linkedin.com/jobs/view/1828401888/,Big Data Engineer,Vodafone,"Madrid, ES",1 day,2020-04-17 23:50,False,"{'yes': ['Portfolio Management', 'Business Int...",NaN,Information Services Information Technology & ...,Full-time,Strategy/Planning,Posted Date\nPosted 1 day ago\nNumber of views...
2,data analyst,Madrid,2020-04-18 23:50,1822423791,https://www.linkedin.com/jobs/view/1822423791/,"Senior Consultant, Data & Analytics (Level 6)",Amazon Web Services (AWS),"Madrid, ES",3 days,2020-04-15 23:50,False,"{'yes': ['Portfolio Management', 'Business Int...",Mid-Senior level,Computer Software Information Technology & Ser...,Full-time,Sales Advertising Business Development,Posted Date\nPosted 3 days ago\nNumber of view...


In [87]:
# ONLY IN CASE TO CORRECT WRONG DF
#df_jobs.drop(columns='Job htmlJob name', inplace=True)

#df_jobs = df_jobs[['JOB TITLE', 'LOCATION', 'SEARCH DATETIME', 'Current Job Id',
#       'Job html', 'Job name', 'Company name', 'Company location',
#       'Posted date', 'Estimated post date', 'Easy apply', 'Skills match',
#       'Seniority Level', 'Industry', 'Employment Type', 'Job Functions',
#       'Job info']]

In [88]:
df_jobs[df_jobs['JOB TITLE']=='data analyst'].shape

(179, 17)

In [89]:
df_jobs[df_jobs['JOB TITLE']=='data scientist'].shape

(392, 17)

In [90]:
df_jobs[df_jobs['JOB TITLE']=='data engineer'].shape

(288, 17)

## Analizyng info

### > Cleaning jobs:

In [91]:
df_jobs.loc[0,'JOB TITLE':'Current Job Id']

JOB TITLE              data analyst
LOCATION                     Madrid
SEARCH DATETIME    2020-04-18 23:50
Current Job Id           1804937837
Name: 0, dtype: object

##### * JOB TITLE_ID

In [92]:
df_jobs['JOB TITLE_ID'] = df_jobs.apply(lambda x: x['JOB TITLE'] +"_"+ str(x['Current Job Id']), axis = 1 )

In [93]:
df_jobs.shape[0] 

859

In [94]:
len(df_jobs['JOB TITLE_ID'].unique())

855

In [95]:
pd.pivot_table(df_jobs,
               values="LOCATION",
               index=['JOB TITLE_ID'],
               aggfunc=np.count_nonzero        
              ).sort_values(by=['LOCATION'], ascending=False)

,LOCATION
JOB TITLE_ID,
data engineer_1774326710,2
data engineer_1806878071,2
data engineer_1384647865,2
data engineer_1696188592,2
data analyst_1298845523,1
...,...
data engineer_1776649341,1
data engineer_1776706802,1
data engineer_1778218609,1


In [96]:
df_jobs[df_jobs['Current Job Id']==1774326710]

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,Easy apply,Skills match,Seniority Level,Industry,Employment Type,Job Functions,Job info,JOB TITLE_ID
572,data engineer,Madrid,2020-04-18 23:50,1774326710,https://www.linkedin.com/jobs/view/1774326710/,Senior Data Engineer,Packlink — Simply Shipping,"Madrid, Comunidad de Madrid, España",1 week,2020-04-11 23:50,True,"{'yes': ['Python (Programming Language)', 'SQL...",Associate,Internet,Full-time,Information Technology,Posted Date\nPosted 1 week ago\nNumber of view...,data engineer_1774326710
597,data engineer,Madrid,2020-04-18 23:50,1774326710,https://www.linkedin.com/jobs/view/1774326710/,Senior Data Engineer,Packlink — Simply Shipping,"Madrid, Comunidad de Madrid, España",1 week,2020-04-11 23:50,True,"{'yes': ['Python (Programming Language)', 'SQL...",Associate,Internet,Full-time,Information Technology,Posted Date\nPosted 1 week ago\nNumber of view...,data engineer_1774326710


In [97]:
df_jobs.loc[572,'Job info'] == df_jobs.loc[597,'Job info']

True

In [98]:
df_jobs.shape[0] - len(df_jobs['JOB TITLE_ID'].unique())

4

In [99]:
df_jobs.drop_duplicates(subset='JOB TITLE_ID', keep='first', inplace = True)

In [100]:
df_jobs.shape

(855, 18)

##### * Current Job Id

In [101]:
len(df_jobs['Current Job Id'].unique())

638

In [102]:
pd.pivot_table(df_jobs,
               values="LOCATION",
               index=['Current Job Id'],
               aggfunc=np.count_nonzero        
              ).sort_values(by=['LOCATION'], ascending=False)

,LOCATION
Current Job Id,
1794127974,3
1828401888,3
1764841143,3
1768494778,3
1757263851,3
...,...
1794130353,1
1793255231,1
1793229309,1


In [103]:
df_jobs[df_jobs['Current Job Id']==1645918674]

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,Easy apply,Skills match,Seniority Level,Industry,Employment Type,Job Functions,Job info,JOB TITLE_ID
39,data analyst,Madrid,2020-04-18 23:50,1645918674,https://www.linkedin.com/jobs/view/1645918674/,Senior Data Analyst & Data Scientist,Ekkiden,"Madrid, ES",4 months,2019-12-28 23:50,False,"{'yes': ['Python (Programming Language)', 'SQL...",Associate,Information Technology & Services,Full-time,Analyst Engineering Information Technology,Posted Date\nPosted 4 months ago\nNumber of vi...,data analyst_1645918674
203,data scientist,Madrid,2020-04-18 23:50,1645918674,https://www.linkedin.com/jobs/view/1645918674/,Senior Data Analyst & Data Scientist,Ekkiden,"Madrid, ES",4 months,2019-12-28 23:50,False,"{'yes': ['Python (Programming Language)', 'Com...",Associate,Information Technology & Services,Full-time,Analyst Engineering Information Technology,Posted Date\nPosted 4 months ago\nNumber of vi...,data scientist_1645918674
601,data engineer,Madrid,2020-04-18 23:50,1645918674,https://www.linkedin.com/jobs/view/1645918674/,Senior Data Analyst & Data Scientist,Ekkiden,"Madrid, ES",4 months,2019-12-28 23:50,False,"{'yes': ['Python (Programming Language)', 'SQL...",Associate,Information Technology & Services,Full-time,Analyst Engineering Information Technology,Posted Date\nPosted 4 months ago\nNumber of vi...,data engineer_1645918674


### > Language of each job description:

In [104]:
df_jobs['Job name'].head(10)

0                         Business Process Analyst
1                                Big Data Engineer
2    Senior Consultant, Data & Analytics (Level 6)
3                      Senior Data Analyst, Madrid
4                                     Data Analyst
5             Junior Business Intelligence Analyst
6                              Data Analyst Junior
7                                     Data Analyst
8                                 Business Analyst
9                            Business Data Analyst
Name: Job name, dtype: object

In [105]:
s = df_jobs.loc[6,'Job info']
s

'Posted Date\nPosted 1 month ago\nNumber of views\n311 views\nShare\nShow more options\nSave\nSave Data Analyst Junior at Tempotel\nApply\nSee how you compare to 51 applicants. Reactivate Premium\nJob\n51 applicants\nAssociate\nCompany\n51-200 employees\nHuman Resources\nConnections\nYou have 0 connections at this company.\nAdd >\nFUNCIONES\n\nMisión: Asegurar que la arquitectura de datos de empleados funciona correctamente y apoya las necesidades de la compañía y de los empleados buscando la excelencia en la calidad de los datos (Carga masiva de ficheros a la aplicación).\n\nFunciones\nAnalista de datos, informes y análisis.\nTrabajar en el Equipo Global en todos los niveles del negocio enfocado principalmente en dar servicio a los Equipos Locales.\nCapacidad para validar un gran número de informes con grandes volúmenes de datos.\nREQUISITOS\n\nFormación: Ingeniería, Grado en Matemáticas, Estadistica, etc.\n\nGran experiencia en Excel, SQL e idealmente SFSF (Sucessfactor)\n\nHerramien

In [106]:
detect(s)

'es'

In [107]:
df_jobs['Info Language'] = df_jobs['Job info'].apply(lambda x: detect(x))

In [108]:
df_jobs['Info Language'].unique()

array(['en', 'es'], dtype=object)

In [109]:
df_jobs['Seniority Level'].unique()

array(['Associate', nan, 'Mid-Senior level', 'Entry level', 'Director',
       'Internship', 'Executive'], dtype=object)

### > Coincidences with definitions:

In [110]:
def coincidencias(job_skills, post_skills):
    c = 0
    for i in job_skills: 
        if re.search(i,post_skills): 
            c+=1
    return c
        

In [111]:
def list_coincidencias(job_skills, post_skills):
    l=[]
    for i in job_skills:
        if re.search(i,post_skills):
            l.append(i)
    return l
        

In [112]:
def clean_text(text):
    text = text.replace('\n',' ')
    text = text.replace('á','a')
    text = text.replace('é','e')
    text = text.replace('í','i')
    text = text.replace('ó','o')
    text = text.replace('ú','u')
    text = text.replace('.',' ')
    text = text.lower()
    
    return text
    

#### jobs >

In [113]:
DA_list = dic_key_words.get('data analyst')
DS_list = dic_key_words.get('data scientist')
DE_list = dic_key_words.get('data engineer')

DA_list

['data analyst',
 'analisis de datos',
 'data analytics',
 'analitica de datos',
 'business analyst',
 'analisis de negocios',
 'sql',
 'python',
 'business intelligence',
 'inteligencia de negocios',
 ' bi ',
 'excel',
 'microsoft office',
 'ms office',
 'dashboard',
 'kpi',
 'reporting',
 'informes',
 'tableau',
 'powerbi',
 'power bi',
 'microsoft bi',
 'qlik',
 'spotfire']

In [114]:
df_jobs['DA coincidences'] = \
df_jobs['Job info'].apply(lambda x: coincidencias(DA_list,clean_text(x)))
df_jobs['DA coincidences_list'] = \
df_jobs['Job info'].apply(lambda x: list_coincidencias(DA_list,clean_text(x)))

df_jobs['DS coincidences'] = \
df_jobs['Job info'].apply(lambda x: coincidencias(DS_list,clean_text(x)))
df_jobs['DS coincidences_list'] = \
df_jobs['Job info'].apply(lambda x: list_coincidencias(DS_list,clean_text(x)))

df_jobs['DE coincidences'] = \
df_jobs['Job info'].apply(lambda x: coincidencias(DE_list,clean_text(x)))
df_jobs['DE coincidences_list'] = \
df_jobs['Job info'].apply(lambda x: list_coincidencias(DE_list,clean_text(x)))

#### profile >

In [119]:
df_profile.loc['total skills', 'info']

"['analytical skills', 'bash', 'business analysis', 'business intelligence', 'business process improvement', 'continuous improvement', 'data analysis', 'data analytics', 'data modeling', 'data pipelines', 'data science', 'data visualization', 'engineering', 'engineering - be', 'english', 'español', 'financial analysis', 'git', 'industrial engineering', 'industry knowledge', 'jupyter', 'leadership', 'lean manufacturing', 'machine learning', 'master', 'microsoft excel', 'microsoft office', 'microsoft power bi', 'microsoft power query', 'microsoft project', 'mysql', 'negotiation', 'numpy', 'pandas', 'power pivot', 'process engineering', 'process optimization', 'programming', 'project management', 'python', 'r', 'sap', 'scrum', 'seaborn', 'six sigma', 'spanish', 'sql', 'statistics', 'tableau', 'team management', 'teamwork', 'telco', 'vba', 'visual basic for applications', 'web scraping']"

In [122]:
#### THIS IS ONLY FOR JUPYTER !!!!!!!!!!!!!!!!!!!!
profile_skills = clean_text(df_profile.loc['skills', 'info']).replace('[','').replace(']','').replace("'","").split(', ')
sorted(profile_skills)

['analytical skills',
 'bash',
 'business analysis',
 'business intelligence',
 'business process improvement',
 'continuous improvement',
 'data analysis',
 'data analytics',
 'data modeling',
 'data pipelines',
 'data science',
 'data visualization',
 'engineering',
 'english',
 'español',
 'financial analysis',
 'git',
 'industry knowledge',
 'jupyter',
 'leadership',
 'lean manufacturing',
 'machine learning',
 'microsoft excel',
 'microsoft office',
 'microsoft power bi',
 'microsoft power query',
 'microsoft project',
 'mysql',
 'negotiation',
 'numpy',
 'pandas',
 'power pivot',
 'process engineering',
 'process optimization',
 'programming',
 'project management',
 'python',
 'r',
 'sap',
 'scrum',
 'seaborn',
 'six sigma',
 'sql',
 'statistics',
 'tableau',
 'team management',
 'teamwork',
 'telco',
 'vba',
 'visual basic for applications',
 'web scraping']

In [123]:
df_jobs['PROFILE coincidences'] = \
df_jobs['Job info'].apply(lambda x:      coincidencias(profile_skills,clean_text(x)))
df_jobs['PROFILE coincidences_list'] = \
df_jobs['Job info'].apply(lambda x: list_coincidencias(profile_skills,clean_text(x)))

#### checking results >

In [124]:
check = 10

In [125]:
df_jobs.loc[check ,'Job name']

'Data Analyst'

In [126]:
dict(df_jobs.loc[check ,'DA coincidences':'PROFILE coincidences_list'])

{'DA coincidences': 6,
 'DA coincidences_list': ['data analyst',
  'sql',
  'business intelligence',
  'excel',
  'kpi',
  'reporting'],
 'DS coincidences': 2,
 'DS coincidences_list': ['sql', 'statistics'],
 'DE coincidences': 1,
 'DE coincidences_list': ['sql'],
 'PROFILE coincidences': 8,
 'PROFILE coincidences_list': ['business intelligence',
  'r',
  'data analysis',
  'english',
  'engineering',
  'statistics',
  'sql',
  'microsoft excel']}

In [127]:
clean_text(df_jobs.loc[check ,'Job info'])

'posted date posted 1 week ago number of views 562 views share show more options save save data analyst at teva pharmaceuticals apply see how you compare to 53 applicants  reactivate premium job 53 applicants entry level company 10001+ employees pharmaceuticals connections you have 0 connections at this company  add > company info  teva is a global pharmaceutical leader and the world\'s largest generic medicines producer, committed to improving health and increasing access to quality health solutions worldwide  our employees are at the core of our success, with colleagues in over 80 countries delivering the world\'s largest medicine cabinet to 200 million people every day  we offer a uniquely diverse portfolio of products and solutions for patients and we\'ve built a promising pipeline centered around our core therapeutic areas  we are continually developing patient-centric solutions and significantly growing both our generic and specialty medicines business through investment in resea

### TOP JOB POST >

In [134]:
top = 5

In [135]:
df_jobs.sort_values(by=['PROFILE coincidences'], ascending=False).head(top)

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,...,JOB TITLE_ID,Info Language,DA coincidences,DA coincidences_list,DS coincidences,DS coincidences_list,DE coincidences,DE coincidences_list,PROFILE coincidences,PROFILE coincidences_list
580,data engineer,Madrid,2020-04-18 23:50,1788328087,https://www.linkedin.com/jobs/view/1788328087/,Data Engineers (0 – 5 years of experience) at ...,EY,"Madrid Area, Spain",1 week,2020-04-11 23:50,...,data engineer_1788328087,en,9,"[data analytics, sql, python, business intelli...",6,"[data science, sql, python, machine learning, ...",8,"[data engineer, sql, no sql, python, java , sp...",18,"[business intelligence, programming, r, englis..."
442,data scientist,Madrid,2020-04-18 23:50,1772483934,https://www.linkedin.com/jobs/view/1772483934/,Data Scientist & Project Administrator (SoF),Brambles,"Madrid, ES",1 month,2020-03-21 23:50,...,data scientist_1772483934,en,8,"[data analytics, sql, python, business intelli...",7,"[data scientist, data science, sql, python, r...",5,"[sql, python, r , hadoop, big data]",16,"[business intelligence, programming, r, englis..."
740,data engineer,Madrid,2020-04-18 23:50,1772483934,https://www.linkedin.com/jobs/view/1772483934/,Data Scientist & Project Administrator (SoF),Brambles,"Madrid, ES",1 month,2020-03-21 23:50,...,data engineer_1772483934,en,8,"[data analytics, sql, python, business intelli...",7,"[data scientist, data science, sql, python, r...",5,"[sql, python, r , hadoop, big data]",16,"[business intelligence, programming, r, englis..."
607,data engineer,Madrid,2020-04-18 23:50,1713962733,https://www.linkedin.com/jobs/view/1713962733/,Data Scientist & Project Manager,CHEP,"Madrid, Community of Madrid, Spain",3 weeks,2020-03-28 23:50,...,data engineer_1713962733,en,8,"[data analytics, sql, python, business intelli...",8,"[data scientist, data science, sql, python, r...",5,"[sql, python, r , hadoop, big data]",15,"[business intelligence, programming, r, englis..."
303,data scientist,Madrid,2020-04-18 23:50,1800986416,https://www.linkedin.com/jobs/view/1800986416/,Data Scientist & Project Administrator,CHEP,"Madrid, ES",4 days,2020-04-14 23:50,...,data scientist_1800986416,en,8,"[data analytics, sql, python, business intelli...",7,"[data scientist, data science, sql, python, r...",5,"[sql, python, r , hadoop, big data]",15,"[business intelligence, programming, r, englis..."


In [130]:
df_jobs[df_jobs['Current Job Id']==1788328087]

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,...,JOB TITLE_ID,Info Language,DA coincidences,DA coincidences_list,DS coincidences,DS coincidences_list,DE coincidences,DE coincidences_list,PROFILE coincidences,PROFILE coincidences_list
580,data engineer,Madrid,2020-04-18 23:50,1788328087,https://www.linkedin.com/jobs/view/1788328087/,Data Engineers (0 – 5 years of experience) at ...,EY,"Madrid Area, Spain",1 week,2020-04-11 23:50,...,data engineer_1788328087,en,9,"[data analytics, sql, python, business intelli...",6,"[data science, sql, python, machine learning, ...",8,"[data engineer, sql, no sql, python, java , sp...",18,"[business intelligence, programming, r, englis..."


## Using NLP

In [78]:
nlp_en = spacy.load('en_core_web_md')
nlp_es = spacy.load('es_core_news_md')

In [131]:
df_jobs.columns

Index(['JOB TITLE', 'LOCATION', 'SEARCH DATETIME', 'Current Job Id',
       'Job html', 'Job name', 'Company name', 'Company location',
       'Posted date', 'Estimated post date', 'Easy apply', 'Skills match',
       'Seniority Level', 'Industry', 'Employment Type', 'Job Functions',
       'Job info', 'JOB TITLE_ID', 'Info Language', 'DA coincidences',
       'DA coincidences_list', 'DS coincidences', 'DS coincidences_list',
       'DE coincidences', 'DE coincidences_list', 'PROFILE coincidences',
       'PROFILE coincidences_list'],
      dtype='object')

In [197]:
unique_jobs = list(df_jobs['JOB TITLE'].unique())
unique_jobs

['data analyst', 'data scientist', 'data engineer']

In [201]:
unique_jobs[0].split(" ")[0][0].upper() + unique_jobs[0].split(" ")[1][0].upper()

'DA'

In [198]:
unique_languages = list(df_jobs['Info Language'].unique())
unique_languages

['en', 'es']

In [199]:
dict_corpus = {}
dict_filtered_tokens = {}

In [217]:
for job in unique_jobs:
    job_key = job.split(" ")[0][0] + job.split(" ")[1][0]
    job_key = job_key.upper()
    
    for lang in unique_languages:
        #CORPUS
        filter_job  = df_jobs['JOB TITLE'] == job
        filter_lang = df_jobs['Info Language'] == lang
        corpus = df_jobs['Job info'][filter_job & filter_lang].str.cat(sep=" || ")
        
        dict_corpus.update( {job_key : corpus} )
        
        if   lang == 'en':
            tokens = nlp_en(corpus)
            STOP_WORDS = SW_en
        elif lang == 'es':
            tokens = nlp_es(corpus)
            STOP_WORDS = SW_es

        filtered_tokens = []

        for word in tokens:
            lemma = word.lemma_.lower().strip()

            if lemma not in STOP_WORDS and re.search('^[a-z]+$', lemma) and lemma not in black_list:
                    filtered_tokens.append(lemma)
        
        dict_filtered_tokens.update( {job_key : filtered_tokens} )

In [218]:
dict_corpus.get('DA')[400:600]

' de la marca en España y por ese motivo estamos buscando un/a Consultor/a con alto conocimiento PowerBI y otras herramientas relacionadas con el análisis de datos. El objetivo será generar, obtener y '

In [219]:
dict_filtered_tokens.get('DA')[400:600]

['legal',
 'seniority',
 'level',
 'associate',
 'marketing',
 'advertising',
 'staffing',
 'recruiting',
 'human',
 'resources',
 'employment',
 'type',
 'full',
 'timar',
 'functions',
 'information',
 'technology',
 'posted',
 'posted',
 'months',
 'number',
 'of',
 'views',
 'views',
 'show',
 'morar',
 'options',
 'business',
 'data',
 'analyst',
 'at',
 'social',
 'you',
 'see',
 'how',
 'you',
 'comparar',
 'to',
 'applicants',
 'applicants',
 'associate',
 'employees',
 'internet',
 'connections',
 'you',
 'have',
 'connections',
 'at',
 'this',
 'business',
 'data',
 'analyst',
 'referencia',
 'madrid',
 'cliente',
 'consultor',
 'especializar',
 'servicio',
 'enfocar',
 'a',
 'calidad',
 'y',
 'seguridad',
 'principal',
 'objetivar',
 'ayudar',
 'a',
 'cliente',
 'a',
 'convertir',
 'departamento',
 'it',
 'motor',
 'productividad',
 'utilizar',
 'adecuar',
 'eficiente',
 'rentable',
 'y',
 'sostenible',
 'disponen',
 'sedar',
 'madrid',
 'y',
 'barcelona',
 'servicio',
 'a',

### Data analyst >

In [148]:
filter_da_1 = df_jobs['JOB TITLE'] == 'data analyst'
filter_da_l1 = df_jobs['Info Language'] == 'en'
filter_da_l2 = df_jobs['Info Language'] == 'es'

In [149]:
df_jobs[filter_da_1 & filter_da_l1].shape

(95, 27)

In [150]:
df_jobs[filter_da_1 & filter_da_l2].shape

(84, 27)

In [152]:
corpus_DA_en = df_jobs['Job info'][filter_da_1 & filter_da_l1].str.cat(sep=" || ")
corpus_DA_es = df_jobs['Job info'][filter_da_1 & filter_da_l2].str.cat(sep=" || ")

In [153]:
len(corpus_DA_en)

369341

In [154]:
len(corpus_DA_es)

208147

In [ ]:
corpus_DA_es[0:5000]

In [192]:
def spacy_tokenizer(corpus, lang):
    
    if   lang == 'en':
        tokens = nlp_en(corpus)
        STOP_WORDS = SW_en
    elif lang == 'es':
        tokens = nlp_es(corpus)
        STOP_WORDS = SW_es
    
    filtered_tokens = []

    for word in tokens:
        lemma = word.lemma_.lower().strip()

        if lemma not in STOP_WORDS and re.search('^[a-z]+$', lemma) and lemma not in black_list:
                filtered_tokens.append(lemma)
    
    return filtered_tokens

In [178]:
black_list = ['post','view','ago','date','option','share','reactivate','premium','company','connection',
              'add', 'save','compare','day','applicant','apply','industry','job',
             ]

#### english:

In [156]:
tokens_DA_en = nlp_en(corpus_DA_en)

In [190]:
tokens_DA_en[100:500]

industry were business challenges and opportunities grow every day.
SWIFT’s Professional Services are a means to help our customers with their challenges to adhere to regulation, reduce costs and risks, streamline and ensure smooth operations and to improve the end customer’s experience. Our goal is to help our customers use SWIFT in the best way possible and excel faster.

The Services Marketing and Engineering team is part of the Services Enablement and Customer operations -Customer Experience organization. Our team enables the services organization to effectively provide high value and experience to our customers; We deliver around the following three main activities 1 - Portfolio management for Professional services, Training and Care/Support; 2 - Data analytics and predictive models; 3 - Business process design, Services delivery methodologies and Risk assessment

We are looking for a Business process analyst who will be responsible for analyzing critical business process with the

In [ ]:
SP_en

In [ ]:
filtered_tokens_DA_en = spacy_tokenizer(corpus_DA_en, 'en')

In [ ]:
filtered_tokens_DA_en[100:500]

#### spanish:

In [193]:
filtered_tokens_DA_es = spacy_tokenizer(corpus_DA_es, 'es')

### Data scientist >

In [72]:
df_jobs[df_jobs['JOB TITLE'] == 'data scientist'].shape

(392, 27)

In [69]:
corpus_DS = df_jobs['JOB TITLE'][df_jobs['JOB TITLE'] == 'data scientist'].str.cat(sep=" || ")

In [70]:
len(corpus_DS)

7052

SyntaxError: invalid syntax (<ipython-input-194-cdb0facc3ef4>, line 1)